In [ ]:
#|default_exp models.ResCNN

# ResCNN

>This is an unofficial PyTorch implementation by Ignacio Oguiza - oguiza@timeseriesAI.co

In [ ]:
#|export
from tsai.imports import *
from tsai.utils import *
from tsai.models.layers import *

In [ ]:
from tsai.models.utils import *

In [ ]:
#|export
class _ResCNNBlock(Module):
    def __init__(self, ni, nf, kss=[7, 5, 3], coord=False, separable=False, zero_norm=False):
        self.convblock1 = ConvBlock(ni, nf, kss[0], coord=coord, separable=separable)
        self.convblock2 = ConvBlock(nf, nf, kss[1], coord=coord, separable=separable)
        self.convblock3 = ConvBlock(nf, nf, kss[2], act=None, coord=coord, separable=separable, zero_norm=zero_norm)

        # expand channels for the sum if necessary
        self.shortcut = ConvBN(ni, nf, 1, coord=coord)
        self.add = Add()
        self.act = nn.ReLU()

    def forward(self, x):
        res = x
        x = self.convblock1(x)
        x = self.convblock2(x)
        x = self.convblock3(x)
        x = self.add(x, self.shortcut(res))
        x = self.act(x)
        return x


class ResCNN(Module):
    def __init__(self, c_in, c_out, coord=False, separable=False, zero_norm=False):
        nf = 64
        self.block1 = _ResCNNBlock(c_in, nf, kss=[7, 5, 3], coord=coord, separable=separable, zero_norm=zero_norm)
        self.block2 = ConvBlock(nf, nf * 2, 3, coord=coord, separable=separable, act=nn.LeakyReLU, act_kwargs={'negative_slope':.2})
        self.block3 = ConvBlock(nf * 2, nf * 4, 3, coord=coord, separable=separable, act=nn.PReLU)
        self.block4 = ConvBlock(nf * 4, nf * 2, 3, coord=coord, separable=separable, act=nn.ELU, act_kwargs={'alpha':.3})
        self.gap = nn.AdaptiveAvgPool1d(1)
        self.squeeze = Squeeze(-1)
        self.lin = nn.Linear(nf * 2, c_out)

    def forward(self, x):
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.block4(x)
        x = self.squeeze(self.gap(x))
        return self.lin(x)

In [ ]:
xb = torch.rand(16, 3, 10)
test_eq(ResCNN(3,2,coord=True, separable=True)(xb).shape, [xb.shape[0], 2])
test_eq(count_parameters(ResCNN(3,2)), 257283)

In [ ]:
ResCNN(3,2,coord=True, separable=True)

ResCNN(
  (block1): _ResCNNBlock(
    (convblock1): ConvBlock(
      (0): AddCoords1d()
      (1): SeparableConv1d(
        (depthwise_conv): Conv1d(4, 4, kernel_size=(7,), stride=(1,), padding=(3,), groups=4, bias=False)
        (pointwise_conv): Conv1d(4, 64, kernel_size=(1,), stride=(1,), bias=False)
      )
      (2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): ReLU()
    )
    (convblock2): ConvBlock(
      (0): AddCoords1d()
      (1): SeparableConv1d(
        (depthwise_conv): Conv1d(65, 65, kernel_size=(5,), stride=(1,), padding=(2,), groups=65, bias=False)
        (pointwise_conv): Conv1d(65, 64, kernel_size=(1,), stride=(1,), bias=False)
      )
      (2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): ReLU()
    )
    (convblock3): ConvBlock(
      (0): AddCoords1d()
      (1): SeparableConv1d(
        (depthwise_conv): Conv1d(65, 65, kernel_size=(3,), stride=(1,), padding=(1,), groups

In [ ]:
check_weight(ResCNN(3,2, zero_norm=True), is_bn)

(array([1., 1., 0., 1., 1., 1., 1.], dtype=float32),
 array([0., 0., 0., 0., 0., 0., 0.], dtype=float32))

In [ ]:
#|eval: false
#|hide
from tsai.export import get_nb_name; nb_name = get_nb_name(locals())
from tsai.imports import create_scripts; create_scripts(nb_name)

<IPython.core.display.Javascript object>

/Users/nacho/notebooks/tsai/nbs/104_models.ResCNN.ipynb saved at 2022-11-09 13:01:28
Correct notebook to script conversion! 😃
Wednesday 09/11/22 13:01:30 CET
